# Smart Wearables -- Bonsai Tree Classification on stream data

## 1. Introduction & Library Imports 

In [1]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
from sklearn.model_selection import train_test_split
from bonsai.base.regtree import RegTree
from bonsai.base.alphatree import AlphaTree
from bonsai.base.c45tree import C45Tree
from bonsai.base.ginitree import GiniTree
from bonsai.base.xgbtree import XGBTree
from bonsai.base.friedmantree import FriedmanTree
from bonsai.ensemble.randomforests import RandomForests
from bonsai.ensemble.paloboost import PaloBoost
from bonsai.ensemble.gbm import GBM
import copy
import sys
import json
import time

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
import math
from keras import optimizers
from utils import *
from model import *
from keras.utils.np_utils import to_categorical
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

#import pydot
#import graphviz
#pydot.find_graphviz = lambda: True
import keras

## 2. Data Gathering

In [5]:
data_input_file = 'data/FNOW/MHEALTH.npz'
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tmp = np.load(data_input_file)
np.load = np_load_old

In [6]:
X = tmp['X']
X = X[:, 0, :, :]
y = tmp['y']
folds = tmp['folds']

In [7]:
n_class = y.shape[1]
y = np.argmax(y, axis=1)
print('Hancrafted Template 2017 {}'.format(data_input_file))

Hancrafted Template 2017 data/FNOW/MHEALTH.npz


## 3. Feature Engineering

In [8]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

def feature_extraction(X):
    #Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(copy.copy(sample))
        features = np.hstack((features, A(copy.copy(sample))))
        features = np.hstack((features, SD(copy.copy(sample))))
        features = np.hstack((features, AAD(copy.copy(sample))))
        features = np.hstack((features, ARA(copy.copy(sample))))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X


## 4. RegTree

In [9]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = RegTree(max_depth=8)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.9704] Recall[0.9583] F1[0.9642] at fold[0]
______________________________________________________
Accuracy[0.9926] Recall[0.9931] F1[0.9930] at fold[1]
______________________________________________________
Accuracy[0.9776] Recall[0.9792] F1[0.9791] at fold[2]
______________________________________________________
Accuracy[0.9925] Recall[0.9792] F1[0.9848] at fold[3]
______________________________________________________
Accuracy[0.9776] Recall[0.9792] F1[0.9726] at fold[4]
______________________________________________________
Accuracy[0.9850] Recall[0.9855] F1[0.9855] at fold[5]
______________________________________________________
Accuracy[0.9624] Recall[0.9508] F1[0.9445] at fold[6]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9788] at fold[7]
______________________________________________________
Accuracy[0.9924] Recall[0.9931] F1[0.9924] at fold[8]
______________________________________________________
Accuracy[0.9773] Re

In [11]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.9805] IC [0.9747, 0.9864]
Mean Recall[0.9775] IC [0.9697, 0.9854]
Mean F1[0.9768] IC [0.9684, 0.9852]
Mean size[0.741]
Mean training time[983.061]
Mean prediction time[0.090]


## 5. XGBTree

In [12]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = XGBTree(max_depth=10,min_samples_split=1,min_samples_leaf=1)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.9852] Recall[0.9722] F1[0.9781] at fold[0]
______________________________________________________
Accuracy[0.9926] Recall[0.9931] F1[0.9930] at fold[1]
______________________________________________________
Accuracy[0.9627] Recall[0.9646] F1[0.9649] at fold[2]
______________________________________________________
Accuracy[0.9925] Recall[0.9792] F1[0.9848] at fold[3]
______________________________________________________
Accuracy[0.9701] Recall[0.9577] F1[0.9630] at fold[4]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9791] at fold[5]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9784] at fold[6]
______________________________________________________
Accuracy[0.9850] Recall[0.9855] F1[0.9857] at fold[7]
______________________________________________________
Accuracy[0.9924] Recall[0.9792] F1[0.9848] at fold[8]
______________________________________________________
Accuracy[0.9848] Re

In [14]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.9820] IC [0.9762, 0.9879]
Mean Recall[0.9774] IC [0.9714, 0.9834]
Mean F1[0.9797] IC [0.9742, 0.9851]
Mean size[0.867]
Mean training time[1282.724]
Mean prediction time[0.162]


## 6. FriedmanTree

In [15]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = FriedmanTree(max_depth=8,min_samples_split=1,min_samples_leaf=1)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.9852] Recall[0.9722] F1[0.9781] at fold[0]
______________________________________________________
Accuracy[0.9926] Recall[0.9931] F1[0.9930] at fold[1]
______________________________________________________
Accuracy[0.9776] Recall[0.9792] F1[0.9791] at fold[2]
______________________________________________________
Accuracy[0.9925] Recall[0.9792] F1[0.9848] at fold[3]
______________________________________________________
Accuracy[0.9403] Recall[0.9293] F1[0.9289] at fold[4]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9791] at fold[5]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9722] at fold[6]
______________________________________________________
Accuracy[0.9850] Recall[0.9855] F1[0.9857] at fold[7]
______________________________________________________
Accuracy[0.9848] Recall[0.9722] F1[0.9775] at fold[8]
______________________________________________________
Accuracy[0.9848] Re

In [16]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.9798] IC [0.9711, 0.9884]
Mean Recall[0.9753] IC [0.9653, 0.9854]
Mean F1[0.9763] IC [0.9661, 0.9866]
Mean size[0.727]
Mean training time[1021.785]
Mean prediction time[0.130]


## 7. PaloBoost

In [17]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = PaloBoost(n_estimators=100,max_depth=5)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.9704] Recall[0.9577] F1[0.9635] at fold[0]
______________________________________________________
Accuracy[0.9704] Recall[0.9722] F1[0.9721] at fold[1]
______________________________________________________
Accuracy[0.9552] Recall[0.9583] F1[0.9568] at fold[2]
______________________________________________________
Accuracy[0.9701] Recall[0.9716] F1[0.9709] at fold[3]
______________________________________________________
Accuracy[0.9701] Recall[0.9710] F1[0.9708] at fold[4]
______________________________________________________
Accuracy[0.9549] Recall[0.9571] F1[0.9560] at fold[5]
______________________________________________________
Accuracy[0.9474] Recall[0.9495] F1[0.9488] at fold[6]
______________________________________________________
Accuracy[0.9774] Recall[0.9779] F1[0.9782] at fold[7]
______________________________________________________
Accuracy[0.9773] Recall[0.9792] F1[0.9788] at fold[8]
______________________________________________________
Accuracy[0.9773] Re

In [18]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.9671] IC [0.9608, 0.9733]
Mean Recall[0.9673] IC [0.9611, 0.9735]
Mean F1[0.9674] IC [0.9613, 0.9735]
Mean size[0.211]
Mean training time[26502.110]
Mean prediction time[4.151]


## 8. GBM

In [21]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = GBM(n_estimators=100,max_depth=5)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.9704] Recall[0.9577] F1[0.9630] at fold[0]
______________________________________________________
Accuracy[0.9852] Recall[0.9861] F1[0.9861] at fold[1]
______________________________________________________
Accuracy[0.9552] Recall[0.9583] F1[0.9563] at fold[2]
______________________________________________________
Accuracy[0.9478] Recall[0.9369] F1[0.9413] at fold[3]
______________________________________________________
Accuracy[0.9701] Recall[0.9571] F1[0.9628] at fold[4]
______________________________________________________
Accuracy[0.9774] Recall[0.9785] F1[0.9726] at fold[5]
______________________________________________________
Accuracy[0.9699] Recall[0.9703] F1[0.9705] at fold[6]
______________________________________________________
Accuracy[0.9774] Recall[0.9779] F1[0.9782] at fold[7]
______________________________________________________
Accuracy[0.9848] Recall[0.9861] F1[0.9858] at fold[8]
______________________________________________________
Accuracy[0.9924] Re

In [22]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.9731] IC [0.9652, 0.9810]
Mean Recall[0.9702] IC [0.9601, 0.9803]
Mean F1[0.9709] IC [0.9619, 0.9799]
Mean size[0.211]
Mean training time[22140.196]
Mean prediction time[6.165]


## 9. Conclusion

In [22]:
from prettytable import PrettyTable    
x = PrettyTable()
x.field_names = ["Model", "Mean Accuracy", "Mean Recall", "Mean F1"]
x.add_row(["Reg Tree", 0.9805,0.9775,0.9768])
x.add_row(["XGB Tree", 0.9820, 0.9774, 0.9797])
x.add_row(["Friedman Tree", 0.9798, 0.9753, 0.9763])
x.add_row(["Palo Boost", 0.9671, 0.9673, 0.9674])
x.add_row(["GBM", 0.9731, 0.9702, 0.9709])

y = PrettyTable()
y.field_names = ["Model", "Mean Accuracy", "Mean Recall", "Mean F1"]
y.add_row(["Mean", 97.65, 0, 0])
print(x)
print(y)

+---------------+---------------+-------------+---------+
|     Model     | Mean Accuracy | Mean Recall | Mean F1 |
+---------------+---------------+-------------+---------+
|    Reg Tree   |     0.9805    |    0.9775   |  0.9768 |
|    XGB Tree   |     0.982     |    0.9774   |  0.9797 |
| Friedman Tree |     0.9798    |    0.9753   |  0.9763 |
|   Palo Boost  |     0.9671    |    0.9673   |  0.9674 |
|      GBM      |     0.9731    |    0.9702   |  0.9709 |
+---------------+---------------+-------------+---------+
+-------+---------------+-------------+---------+
| Model | Mean Accuracy | Mean Recall | Mean F1 |
+-------+---------------+-------------+---------+
|  Mean |     97.65     |      0      |    0    |
+-------+---------------+-------------+---------+
